<a href="https://colab.research.google.com/github/HYUNSOOLEE-6839/colab-Deep-Learning/blob/main/41_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94%ED%8F%89_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 네이버 영화 리뷰 감성 분류
- LSTM 활용

In [4]:
# Konlpy 설치
!pip install Konlpy

     |████████████████████████████████| 19.4MB 159kB/s 
     |████████████████████████████████| 92kB 15.5MB/s 
     |████████████████████████████████| 460kB 58.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [5]:
import konlpy
konlpy.__version__

'0.5.2'

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
import numpy as np

In [8]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 파일 업로드

In [9]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving ratings_train.txt to ratings_train.txt


In [10]:
from google.colab import files

uploaded = files.upload()
testname = list(uploaded.keys())[0]

Saving ratings_test.txt to ratings_test.txt


### 데이터 전처리

In [11]:
import pandas as pd
train_df = pd.read_table(filename)
test_df = pd.read_table(testname)

##### 중복여부 확인

In [12]:
train_df['document'].nunique()

146182

##### 중복샘플제거

In [13]:
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

##### Null 값 확인

In [14]:
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

##### Null 값 제거

In [15]:
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

##### 긍정, 부정 레이블 값의 갯수

In [16]:
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

##### 테스트 데이터셋에도 마찬가지로 적용

In [17]:
# 중복 제거
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

(49158, 3)

In [18]:
# Null값 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

### 한글 텍스트 전처리

In [19]:
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [20]:
train_df['document'].replace('', np.nan, inplace=True)
train_df.isnull().sum()

id            0
document    391
label         0
dtype: int64

In [21]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

### 테스트 셋에도 적용

In [22]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]","")
test_df.head(3)

,id,document,label
0,6270596,굳ㅋ,1
1,9274899,,0
2,8544678,뭐야이평점들은나쁘진않지만점짜리는더더욱아니잖아,0


In [23]:
test_df['document'].replace('', np.nan, inplace=True)
test_df.isnull().sum()

id            0
document    305
label         0
dtype: int64

In [24]:
test_df = test_df.dropna(how='any')
test_df.shape

(48852, 3)

### 토큰화와 불용어 제거

In [25]:
from konlpy.tag import Okt

In [26]:
import tqdm.notebook as tn
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
okt = Okt()

In [27]:
X_train=[]
for sentence in tn.tqdm(train_df['document']):
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_train.append(temp_X)

In [28]:
X_test=[]
for sentence in tn.tqdm(test_df['document']):
  temp_X = []
  temp_X = okt.morphs(sentence, stem=True) # 토큰화
  temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
  X_test.append(temp_X)

In [29]:
# 정수 인코딩
max_words = 35000
tokenizer = Tokenizer(num_words=max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [30]:
# 전체 데이터의 길이 분포
print('리뷰의 최대 길이:', max(len(s) for s in X_train))
print('리뷰의 평균 길이:', sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이: 69
리뷰의 평균 길이: 10.911133060339802


In [31]:
import matplotlib.pyplot as plt
%matplotlib inline

In [32]:
# X_train과 X_test의 모든 샘플 길이를 동일하게 30으로 셋팅
max_len = 30
# 전체 데이터의 길이는 30으로 맞춘다.
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [33]:
y_train = train_df['label'].values
y_test = test_df['label'].values

### LSTM 모델 정의/설정/학습/평가

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [38]:
model = Sequential([
    Embedding(max_words, 100),
    LSTM(128),
    Dense(1, activation='sigmoid')
])
model.summary()
# 이진분류이기때문에 sigmoid 사용

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3500000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,617,377
Trainable params: 3,617,377
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='naver-lstm=best=model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)

In [41]:
history = model.fit(X_train, y_train, epochs=4, batch_size=60, validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/4
1944/1944 [==============================] - 76s 34ms/step - loss: 0.4371 - accuracy: 0.7915 - val_loss: 0.3494 - val_accuracy: 0.8473

Epoch 00001: val_loss improved from inf to 0.34942, saving model to naver-lstm=best=model.h5
Epoch 2/4
1944/1944 [==============================] - 65s 33ms/step - loss: 0.2828 - accuracy: 0.8815 - val_loss: 0.3411 - val_accuracy: 0.8528

Epoch 00002: val_loss improved from 0.34942 to 0.34115, saving model to naver-lstm=best=model.h5
Epoch 3/4
1944/1944 [==============================] - 64s 33ms/step - loss: 0.2240 - accuracy: 0.9097 - val_loss: 0.3799 - val_accuracy: 0.8493

Epoch 00003: val_loss did not improve from 0.34115
Epoch 4/4
1944/1944 [==============================] - 64s 33ms/step - loss: 0.1762 - accuracy: 0.9288 - val_loss: 0.4259 - val_accuracy: 0.8453

Epoch 00004: val_loss did not improve from 0.34115


In [43]:
# Best Model 선택
from tensorflow.keras.models import load_model
best_model = load_model('naver-lstm=best=model.h5')

In [45]:
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f"Accuracy:{acc:.4f}")

1527/1527 - 4s - loss: 0.3586 - accuracy: 0.8417
Accuracy:0.8417


In [46]:
# 테스트셋
y_vloss = history.history['val_loss']
y_vacc = history.history['val_accuracy']
# 학습셋
y_loss =history.history['loss']
y_acc = history.history['accuracy']

In [ ]:
x_len = np.arange(1, len(y_loss)+1)
fig, ax0 = plt.subplots(figsize=(10,8))
ax1 = ax0.twinx()
ax0.plot(x_len, y_loss, c='blue', label='Trainset_')
ax0.plot(x_len, y_vloss, c='red', label='Testset_'
ax0.set_ylabel('loss')
ax1.plot(x_len, y_acc, c='darkgreen', label='Trainset_')
ax1.plot(x_len, y_vacc, c='magenta', label='Testset_'
ax1.set_ylabel('accuracy')
ax0.set_xlabel('epoch')
ax0.legend(loc='lower center')
ax1.legend(loc='upper center')
plt.grid()
plt.show()

### Simple RNN

In [47]:
from tensorflow.keras.layers import SimpleRNN
model2 = Sequential([
    Embedding(max_words, 100),
    SimpleRNN(128),
    Dense(1, activation='sigmoid')
])
model2.summary()
# 이진분류이기때문에 sigmoid 사용

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         3500000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               29312     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 3,529,441
Trainable params: 3,529,441
Non-trainable params: 0
_________________________________________________________________


In [49]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='naver-rnn=best=model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)

In [51]:
history = model2.fit(X_train, y_train, epochs=4, batch_size=60, validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/4
1944/1944 [==============================] - 101s 51ms/step - loss: 0.4647 - accuracy: 0.7658 - val_loss: 0.3716 - val_accuracy: 0.8393

Epoch 00001: val_loss improved from inf to 0.37160, saving model to naver-rnn=best=model.h5
Epoch 2/4
1944/1944 [==============================] - 99s 51ms/step - loss: 0.3114 - accuracy: 0.8705 - val_loss: 0.3750 - val_accuracy: 0.8382

Epoch 00002: val_loss did not improve from 0.37160
Epoch 3/4
1944/1944 [==============================] - 100s 51ms/step - loss: 0.2241 - accuracy: 0.9106 - val_loss: 0.4326 - val_accuracy: 0.8170

Epoch 00003: val_loss did not improve from 0.37160
Epoch 4/4
1944/1944 [==============================] - 99s 51ms/step - loss: 0.1647 - accuracy: 0.9367 - val_loss: 0.5292 - val_accuracy: 0.8076

Epoch 00004: val_loss did not improve from 0.37160


In [65]:
best_model2 = load_model('naver-rnn=best=model.h5')

In [66]:
acc2 = best_model2.evaluate(X_test, y_test, verbose=2)[1]
print(f"Accuracy:{acc2:.4f}")

1527/1527 - 5s - loss: 0.3852 - accuracy: 0.8324
Accuracy:0.8324


### LSTM + CNN

In [59]:
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D

In [60]:
model3 = Sequential([
    Embedding(max_words, 100),
    Dropout(0.5),
    Conv1D(64, 5, padding='valid', activation='relu'),
    MaxPooling1D(pool_size=4),
    LSTM(60),
    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         3500000   
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60)                30000     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 3,562,125
Trainable params: 3,562,125
Non-trainable params: 0
____________________________________________

In [61]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='naver-rnn-lstm=best=model.h5',
                               monitor='val_loss', verbose=1, save_best_only=True)
history3 = model3.fit(X_train, y_train, epochs=4, batch_size=60, validation_split=0.2, verbose=1, callbacks=[checkpointer])

Epoch 1/4
1944/1944 [==============================] - 67s 33ms/step - loss: 0.5363 - accuracy: 0.7053 - val_loss: 0.4475 - val_accuracy: 0.7758

Epoch 00001: val_loss improved from inf to 0.44753, saving model to naver-rnn-lstm=best=model.h5
Epoch 2/4
1944/1944 [==============================] - 64s 33ms/step - loss: 0.4010 - accuracy: 0.8073 - val_loss: 0.4412 - val_accuracy: 0.7824

Epoch 00002: val_loss improved from 0.44753 to 0.44121, saving model to naver-rnn-lstm=best=model.h5
Epoch 3/4
1944/1944 [==============================] - 65s 33ms/step - loss: 0.3517 - accuracy: 0.8325 - val_loss: 0.4593 - val_accuracy: 0.7776

Epoch 00003: val_loss did not improve from 0.44121
Epoch 4/4
1944/1944 [==============================] - 64s 33ms/step - loss: 0.3116 - accuracy: 0.8517 - val_loss: 0.4854 - val_accuracy: 0.7769

Epoch 00004: val_loss did not improve from 0.44121


In [64]:
best_model3 = load_model('naver-rnn-lstm=best=model.h5')
acc3 = best_model3.evaluate(X_test, y_test, verbose=2)[1]
print(f"Accuracy:{acc3:.4f}")

1527/1527 - 3s - loss: 0.4602 - accuracy: 0.7680
Accuracy:0.7680
